#### Overview

manipulating tables q-sql, since many of them resemble their SQL counterparts in form or function. The usual suspects such as insert, select, update, are present, as well as functionality that is not available in traditional SQL. But appearances can be deceiving: there are some significant differences in the syntax and behavior.

The first important difference is that a q table has ordered rows and columns.

A second difference is that a q table is stored physically as a collection of column lists. This means that operations on column data are vector operations. Moreover, for simple column lists, atomic, aggregate and uniform functions applied to columns are especially simple and fast since they reduce to direct memory addressing.

#### Inserting Records

Generic inseration
there are a few way
1. targeted with fields
2. naked based on position

you can insert rows into the table,t via { ,: }

In [45]:
t:([] name:`symbol$(); iq:`int$())

//You can insert with fields
t,:`name`iq!(`Beebledrox;42)
t,:`name`iq!(`Dent,92)

//you can insert without as well
t,:(`dog,123)
t,:(`cat,111)
t,:(`mouse,222)

In [46]:
t

name       iq 
--------------
Beebledrox 42 
Dent       92 
dog        123
cat        111
mouse      222


##### Key table
this is like sql where you can have a key value in the table
which means that there can only be 1 unique value for that column

in this case we have eid as the key

In [9]:
kt:([eid:`long$()] name:`symbol$(); iq:`long$())
kt,:(1002; `Dent; 98)

In [5]:
kt

eid | name iq
----| -------
1002| Dent 98


In [7]:
kt,:(1002; `Dent; 101)

In [8]:
kt

eid | name iq 
----| --------
1002| Dent 101


#### Basic Insertation

the function insert -> ,: works in the same way to insert

In [68]:
t:([] name:`Dent`Dan`dant; iq:1 2 3)
t

name iq
-------
Dent 1 
Dan  2 
dant 3 


In [69]:
`t insert (`name`iq)!(`danny;4)
`t insert (`marvin;5)

,3


,4


In [70]:
t

name   iq
---------
Dent   1 
Dan    2 
dant   3 
danny  4 
marvin 5 


In [71]:
//Truncation of table
t:3#t

In [72]:
//You are insert a table into an existing table
`t insert ([] name:`sally`jane;iq:6 9)

3 4


In [73]:
t

name  iq
--------
Dent  1 
Dan   2 
dant  3 
sally 6 
jane  9 


#### Bulk insertation of table

It is also possible to bulk insert naked field values but there is a twist. To bulk insert naked field values, you provide a list of columns not a list of rows.

In [77]:
t:([] name:`symbol$(); iq:`int$())
//t:([] name:(); iq:())

In [75]:
`t insert (`james`norman; 123 234)

0 1


In [76]:
t

name   iq 
----------
james  123
norman 234


#### Insert and Foreign Keys

When inserting data into a table that has foreign key(s), the values destined for the foreign key column(s) are checked to ensure that they appear in the primary key column(s) pointed to by the foreign key(s).

for you to insert anything into tdetails there must exist in kt already

In [1]:
kt:([eid:1001 1002 1003] name:`james`norman`jian; iq: 100 200 300)
ktdetails:([] eid:`kt$1003 1002 1001 1002 1001; sc:126 36 92 39 98)

In [2]:
`ktdetails insert( 1002; 42)

,5


In [2]:
`ktdetails insert( 1042; 11)

[0;31mcast[0m: [0;31mcast[0m

##### Inserting into key tables

same as the previous using the insert command

In [3]:
`kt insert(1004; `tim; 400)

,3


In [4]:
`kt insert(1005; `toom; 100)

,4


**Inserting into a table that already has the key will fail** 

In [4]:
`kt insert(1004;`johh;321)

[0;31minsert[0m: [0;31minsert[0m

**Upsert**

Upsert is similar to insert but better?

1. insert only allows pass by name
2. upsert supports both pass-by-name and pass-by-value

Lets give this a shot

In [7]:
([] c1:`a`b; c2:10 20) upsert(`c; 30)

c1 c2
-----
a  10
b  20
c  30


In [8]:
f:{t:([] c1:`a`b; c2:1 2); t upsert x}

In [9]:
f(`c; 30)

c1 c2
-----
a  1 
b  2 
c  30


**Ooo** 

this is interesting okay so right now f is a function, the first input to the function is given the name x.

